In [2]:
!pip install trulens_eval llama_index llama-index-llms-openai llama_hub llmsherpa

In [6]:
# Import the os module to interact with the operating system
import os

# Import the load_dotenv function from the dotenv module
from dotenv import load_dotenv

# Call the load_dotenv function to load environment variables from a .env file
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [7]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from trulens_eval import Tru
tru = Tru()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [5]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.29.83:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Start with basic RAG.

In [9]:
import glob

In [8]:
from langchain_community.document_loaders import PyPDFLoader

In [15]:
documents = []
for file in glob.glob("/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data" + "/*.pdf"):
    print(file)
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    documents.extend(pages)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/exhibit-13.pdf
/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/third-quarter-10q.pdf
/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/second-quarter-10q.pdf


In [20]:
type(documents[0])

langchain_core.documents.base.Document

In [24]:
type(documents[0].page_content)

str

In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()

In [27]:
documents

[Document(id_='17731d6b-c821-43a5-a2e2-b3c683c624fb', embedding=None, metadata={'page_label': '1', 'file_name': 'exhibit-13.pdf', 'file_path': '/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/exhibit-13.pdf', 'file_type': 'application/pdf', 'file_size': 10182327, 'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Exhibit 13 \nFinancial Review \n4 Overview 110 4 Equity Securities \n9 Earnings Performance 112 5 Loans and Related Allowance for Credit Losses \n27 Balance Sheet Analysis 127 6 Mortgage Banking Activities \n29 Off-Balance Sheet Arrangements 129 7 Intangible Assets and Other Assets \n30 Risk Management 130 8 Leasing Activity \n52 Capital Management 132 9 Deposits \n58 Regula

In [31]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [35]:
from llama_index.legacy import ServiceContext
from llama_index.legacy import VectorStoreIndex, StorageContext, Document
from langchain_community.llms import Ollama

# initialize llm
llm = Ollama(model="llama2", temperature=0)

# # knowledge store
document = Document(text="\n\n".join([doc.text for doc in documents]))
print(document)

# service context for index
service_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

# create index
index = VectorStoreIndex.from_documents([document], service_context=service_context)

Doc ID: 8640fbc7-6e35-4548-a6f9-d73e36431e45
Text: Exhibit 13  Financial Review  4 Overview 110 4 Equity Securities
9 Earnings Performance 112 5 Loans and Related Allowance for Credit
Losses  27 Balance Sheet Analysis 127 6 Mortgage Banking Activities
29 Off-Balance Sheet Arrangements 129 7 Intangible Assets and Other
Assets  30 Risk Management 130 8 Leasing Activity  52 Capital
Management 132...


ImportError: cannot import name 'Prompt' from 'llama_index' (unknown location)

In [36]:
from llama_index.legacy import Prompt

system_prompt = Prompt("We have provided context information below that you may use. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Please answer the question: {query_str}\n")

# basic rag query engine
rag_basic = index.as_query_engine(text_qa_template = system_prompt)

## Load test set

In [37]:
honest_evals = [
    "What are the typical coverage options for homeowners insurance?",
    "What are the requirements for long term care insurance to start?",
    "Can annuity benefits be passed to beneficiaries?",
    "Are credit scores used to set insurance premiums? If so, how?",
    "Who provides flood insurance?",
    "Can you get flood insurance outside high-risk areas?",
    "How much in losses does fraud account for in property & casualty insurance?",
    "Do pay-as-you-drive insurance policies have an impact on greenhouse gas emissions? How much?",
    "What was the most costly earthquake in US history for insurers?",
    "Does it matter who is at fault to be compensated when injured on the job?"
]

## Set up Evaluation

In [39]:
import numpy as np
from trulens_eval import Tru, Feedback, TruLlama, OpenAI as fOpenAI

tru = Tru()

# start fresh
tru.reset_database()

from trulens_eval.feedback import Groundedness

openai = fOpenAI()

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

# embedding distance
from langchain.embeddings.openai import OpenAIEmbeddings
from trulens_eval.feedback import Embeddings

model_name = 'text-embedding-ada-002'

embed_model = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

embed = Embeddings(embed_model=embed_model)
f_embed_dist = (
    Feedback(embed.cosine_distance)
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
)

from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=openai)

f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text.collect())
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)

honest_feedbacks = [qa_relevance, qs_relevance, f_embed_dist, f_groundedness]

from trulens_eval import FeedbackMode

tru_recorder_rag_basic = TruLlama(
        rag_basic,
        app_id='1) Basic RAG - Honest Eval',
        feedbacks=honest_feedbacks
    )

ImportError: cannot import name 'Ollama' from 'trulens_eval' (/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/trulens_eval/__init__.py)

In [ ]:
tru.run_dashboard()

In [ ]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_basic as recording:
    for question in honest_evals:
        response = rag_basic.query(question)

In [ ]:
tru.get_leaderboard(app_ids=["1) Basic RAG - Honest Eval"])

Our simple RAG often struggles with retrieving not enough information from the insurance manual to properly answer the question. The information needed may be just outside the chunk that is identified and retrieved by our app.